In [1]:
#https://github.com/uvipen/Character-level-cnn-pytorch

In [1]:
colab = False
useCuda = False

In [2]:
path = ''
if colab:
    from google.colab import drive
    drive.mount('/content/drive/')
    path = '/content/drive/MyDrive/Term 4.2/IS706 - Software Mining and Analysis/Project/'

In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import sys
import csv
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import shutil
import unicodedata
import string
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
import torch.optim as optim

In [4]:
pd.read_csv(path + 'dataset-cnn/cleaned/training.csv', header=None, sep=';')

,0,1
0,1,com.github.tomakehurst.wiremock.client.Request...
1,1,org.apache.solr.client.solrj.request.LukeReque...
2,1,org.jmock.Expectations.will(Action)__XX__org.j...
3,1,org.elasticsearch.action.index.IndexResponse.v...
4,1,org.mockito.Matchers.anyChar()__XX__org.mockit...
...,...,...
2551,0,com.tencent.angel.ml.core.conf.MLConf.ML_GBDT_...
2552,0,com.tencent.angel.ml.core.conf.MLConf.ML_GBDT_...
2553,0,com.tencent.angel.ml.core.conf.MLConf.ML_GBDT_...
2554,0,com.tencent.angel.ml.core.conf.MLConf.ML_NUM_C...


In [5]:
class MyDataset(Dataset):
    def __init__(self, file, maxl=200, transform=None, target_transform=None):
        self.vocabulary = list("""abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}""")
        self.identity_mat = np.identity(len(self.vocabulary))
        self.arr = pd.read_csv(file, sep=';', header=None)
        self.transform = transform
        self.target_transform = target_transform
        self.max_length = maxl

    def __len__(self):
        return len(self.arr)

    def __getitem__(self, idx):
        label = self.arr.iloc[idx, 0]
        line = self.arr.iloc[idx, 1]
        
        line = np.array([self.identity_mat[self.vocabulary.index(i)] for i in list(line) if i in self.vocabulary],
                        dtype=np.float32)
        
        if len(line) > self.max_length:
            line = line[:self.max_length]
        elif 0 < len(line) < self.max_length:
            line = np.concatenate(
                (line, np.zeros((self.max_length - len(line), len(self.vocabulary)), dtype=np.float32)))
        elif len(line) == 0:
            line = np.zeros((self.max_length, len(self.vocabulary)), dtype=np.float32)
            
    
        return label, line
    

In [6]:
training_dataset = MyDataset(path + 'dataset-cnn/cleaned/training.csv')
validation_dataset = MyDataset(path + 'dataset-cnn/cleaned/validation.csv')

In [7]:
training_dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=64, shuffle=True)

print(len(training_dataloader), len(validation_dataloader))

40 3


In [8]:
dataiter = iter(training_dataloader)
labels, lines = dataiter.next()

print(type(labels))
print(type(lines))

<class 'torch.Tensor'>
<class 'torch.Tensor'>


In [9]:
lines.shape

torch.Size([64, 200, 68])

In [10]:
def get_evaluation(y_true, y_prob, list_metrics):
    y_pred = np.argmax(y_prob, -1)
    output = {}
    if 'accuracy' in list_metrics:
        output['accuracy'] = metrics.accuracy_score(y_true, y_pred)
    if 'loss' in list_metrics:
        try:
            output['loss'] = metrics.log_loss(y_true, y_prob)
        except ValueError:
            output['loss'] = -1
    if 'confusion_matrix' in list_metrics:
        output['confusion_matrix'] = str(metrics.confusion_matrix(y_true, y_pred))
    return output

In [11]:
class CharacterLevelCNN(nn.Module):
    def __init__(self, input_dim = 68, input_len = 200, filters = 256):
        super().__init__()
#         self.conv1 = nn.Conv1d(200, 200, 15)
# #         self.pool = nn.MaxPool1d(2, 2)
#         self.conv2 = nn.Conv1d(200, 200, 15)
#         self.fc1 = nn.Linear(2400, 1200)
#         self.fc2 = nn.Linear(1200, 840)
#         self.fc3 = nn.Linear(840, 2)
    
        self.conv1 = nn.Conv1d(input_dim, filters, 15)
        self.conv2 = nn.Conv1d(filters, filters, 13)
        self.conv3 = nn.Conv1d(filters, filters, 10)
        self.conv4 = nn.Conv1d(filters, filters, 10)
        self.fc1 = nn.Linear(39936, 30000)
        self.fc2 = nn.Linear(30000, 20000)
        self.fc3 = nn.Linear(20000, 12000)
        self.fc4 = nn.Linear(12000, 6000)
        self.fc5 = nn.Linear(6000, 1000)
        self.fc6 = nn.Linear(1000, 2)
    
    def forward(self, x):
        x = x.transpose(1, 2)
#         print("input: ", x.shape)
        
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
#         print("shape after convolution:", x.shape)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        # print("shape after flattenning:", x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [12]:
model = CharacterLevelCNN()

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.3)

In [14]:
if torch.cuda.is_available() and useCuda == True:
    print("Using Cuda")
    model.cuda()


In [15]:
model.train()

CharacterLevelCNN(
  (conv1): Conv1d(68, 256, kernel_size=(15,), stride=(1,))
  (conv2): Conv1d(256, 256, kernel_size=(13,), stride=(1,))
  (conv3): Conv1d(256, 256, kernel_size=(10,), stride=(1,))
  (conv4): Conv1d(256, 256, kernel_size=(10,), stride=(1,))
  (fc1): Linear(in_features=39936, out_features=30000, bias=True)
  (fc2): Linear(in_features=30000, out_features=20000, bias=True)
  (fc3): Linear(in_features=20000, out_features=12000, bias=True)
  (fc4): Linear(in_features=12000, out_features=6000, bias=True)
  (fc5): Linear(in_features=6000, out_features=1000, bias=True)
  (fc6): Linear(in_features=1000, out_features=2, bias=True)
)

In [16]:
num_epochs = 30
for epoch in range(num_epochs):
        for i, batch in enumerate(training_dataloader):
            labels, lines = batch
            if useCuda and torch.cuda.is_available():
                lines = lines.cuda()
                labels = labels.cuda()
            optimizer.zero_grad()
            predictions = model(lines)
            loss = criterion(predictions, labels)
            loss.backward()
            optimizer.step()

            training_metrics = get_evaluation(labels.cpu().numpy(), predictions.cpu().detach().numpy(),
                                              list_metrics=["accuracy"])
            print("Epoch: {}/{}, Iteration: {}/{}, Lr: {}, Loss: {}, Accuracy: {}".format(
                epoch + 1,
                num_epochs,
                i + 1,
                len(training_dataloader),
                optimizer.param_groups[0]['lr'],
                loss, training_metrics["accuracy"]))
        model.eval()
        loss_ls = []
        te_label_ls = []
        te_pred_ls = []
        for batch in validation_dataloader:
            te_label, te_feature = batch
            num_sample = len(te_label)
            if torch.cuda.is_available():
                te_feature = te_feature.cuda()
                te_label = te_label.cuda()
            with torch.no_grad():
                te_predictions = model(te_feature)
            te_loss = criterion(te_predictions, te_label)
            loss_ls.append(te_loss * num_sample)
            te_label_ls.extend(te_label.clone().cpu())
            te_pred_ls.append(te_predictions.clone().cpu())


KeyboardInterrupt: 